# Multi-Agent System for Workshop Preparation

This notebook demonstrates how to build a multi-agent system using [smolagents](https://huggingface.co/docs/smolagents), where each agent performs a specialized task. The manager agent orchestrates their collaboration to assist users in preparing for datacraft workshops based on their learning goals.

## 1. Load dependencies and environment

We start by importing required packages and loading environment variables (such as the OpenAI API key).

In [2]:
import os
from dotenv import load_dotenv
from smolagents import CodeAgent, OpenAIServerModel

from workshop_smolagents_mcp.tools import query_events_db, web_search, summarize_url

load_dotenv(override=True)

/Users/raphael/workshop-smolagents-mcp/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## 2. Initialize the language model

We configure the OpenAI model that will be used across all agents.

In [3]:
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

## 3. Create specialized agents

We create two agents:
- `WebAgent` can search and summarize content from the web.
- `EventsAgent` can query a local SQLite database of datacraft workshops.

In [ ]:
web_agent = CodeAgent(
    tools=[web_search, summarize_url],
    model=model,
    name="WebAgent",
    description="Can search the web and find urls of online sources and summarize the content of the urls.",
)

events_agent = CodeAgent(
    tools=[query_events_db],
    model=model,
    name="EventsAgent",
    description="Can query the events database to find relevant datacraft events based on user mention of an existing event.",
)

## 4. Create the manager agent

This agent coordinates the other two. It receives user goals and decides how to split the task between the database and web agents.

In [7]:
manager_agent = CodeAgent(
    tools=[],  # Required, even if empty
    model=model,
    managed_agents=[web_agent, events_agent],
    name="WorkshopPreparationAgent",
    description=(
        "An intelligent assistant that helps datacraft event organizers draft a workshop invitation for members."
        "Given a user's workshop mention (e.g. 'I want to invite members for the upcoming workshop on watermarking'), it finds the relevant workshop "
        "from the internal database and supplements this data with external learning resources from the web that can be used to prepare the workshop."
    ),
)

## 5. Run an example request

We simulate a user asking to improve their skills in generative AI. The manager agent will find both workshop and web resources to match.

In [8]:
user_goal = "Help me draft an invitation for the event which nomenclature is 2506-Geodis-GenAI4dev"
response = manager_agent.run(user_goal)
print(response)

╭────────────────────────────────────── New run - WorkshopPreparationAgent ───────────────────────────────────────╮
│                                                                                                                 │
│ Help me draft an invitation for the event which nomenclature is 2506-Geodis-GenAI4dev                           │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  event_info = EventsAgent(task="Provide details for the event with the nomenclature 2506-Geodis-GenAI4dev.")      
  print(event_info)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────────────── New run - EventsAgent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'EventsAgent'.                                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Provide details for the event with the nomenclature 2506-Geodis-GenAI4dev.                                      │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            Thought: I will address the persistent issue by providing the completion message in a thought sequence 
instead of enclosing it in a code block, as it should not be interpreted as Python code.

The invitation draft has been correctly formatted and shared. Please customize it by replacing placeholders such as
[Recipient's Name\], [Your Name\], [Your Position\], and [Your Contact Information\] with specific details. If you 
have additional questions or require further assistance, please feel free to ask.</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 20: Duration 2.92 seconds| Input tokens: 103,868 | Output tokens: 1,973]

Reached max steps.

[Step 21: Duration 7.58 seconds| Input tokens: 109,878 | Output tokens: 2,224]

To create an effective invitation for your event, I'll include all the necessary details based on the information we have:

---

**Subject: Invitation to REX: Deploying AI Coding Assistants in Enterprise**

Dear [Recipient's Name],

We are excited to invite you to attend an insightful event titled "REX: How we deployed AI coding assistants to 80+ developers in a large enterprise," scheduled for **17th September 2025**, from **19h to 21h**.

During this event, we will share our experiences and challenges faced while implementing AI coding assistants for over 80 developers in a large enterprise. The session will cover adoption strategies, results, gains, and valuable lessons learned. We'll also discuss future scaling opportunities.

The event will be conducted in **French** and is tagged with #GenAI, #GenerativeAI, #DeveloperExperience, and #REX.

Don't miss out on this chance to explore the transformative impact of AI in development with industry experts.

We look forward to your partic